In [26]:
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np
import pandas as pd

with open('./데이터만 일단 모아둠/SentiWord_Dict.txt', 'r', encoding='utf-8') as f:
    text = f.read()

#x,y값을 추출후 데이터프레임에 넣는다.
# X 값 추출 피쳐feature, y값 추출 target
df_train = pd.DataFrame({
    'feature': [line.split('\t')[0] for line in text.split('\n') if len(line.split('\t')) >= 2],
    'target': [int(line.split('\t')[1]) for line in text.split('\n') if len(line.split('\t')) >= 2]
})


In [27]:
df_train  #14856 rows × 2 columns

,feature,target
0,(-;,1
1,(;_;),-1
2,(^^),1
3,(^-^),1
4,(^^*,1
...,...,...
14851,내팽개치다,-2
14852,횡령,-2
14853,불안증,-2
14854,어쩔티비,-1


In [30]:
#널값 제거
df_train[df_train['feature'].isnull()].count()
df_train #14856 rows × 2 columns

#중복값이 있는 줄 제거 => 하나 제거된듯
df_train.drop_duplicates()  #14855 rows × 2 columns

,feature,target
0,(-;,1
1,(;_;),-1
2,(^^),1
3,(^-^),1
4,(^^*,1
...,...,...
14851,내팽개치다,-2
14852,횡령,-2
14853,불안증,-2
14854,어쩔티비,-1


In [31]:
df_train['target'].value_counts()
#데이터가 불균형하다.

-1    5031
-2    4799
 2    2603
 1    2269
 0     154
Name: target, dtype: int64

In [32]:
# print(text)

In [33]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder 

#LabelEncoder 객체를 생성
label_encoder = LabelEncoder()

#타겟 변수를 인코딩
encoded_target = label_encoder.fit_transform(df_train['target'])

#인코딩된 타겟 변수를 원래 데이터프레임에 추가하거나 교체합니다.
df_train['encoded_target'] = encoded_target

#인코딩된 라벨을 다시 점수값으로 되돌리는 코드
decoded_target = label_encoder.inverse_transform(encoded_target)

# 라벨별로 분류
very_bad = df_train[df_train['target'] == -2]
bad   =  df_train[df_train['target'] == -1]
soso = df_train[df_train['target'] == 0]
good  = df_train[df_train['target'] == 1]
very_good = df_train[df_train['target'] == 2]

In [35]:
#인코딩된 라벨을 다시확인
df_train['encoded_target'].value_counts()


1    5031
0    4799
4    2603
3    2269
2     154
Name: encoded_target, dtype: int64

In [36]:
very_bad

,feature,target,encoded_target
49,가난,-2,0
50,가난뱅이,-2,0
51,가난살이,-2,0
52,가난살이하다,-2,0
53,가난설음,-2,0
...,...,...,...
14848,실점,-2,0
14849,오류,-2,0
14851,내팽개치다,-2,0
14852,횡령,-2,0


In [64]:
# 예시로 상위 10000개 데이터만 사용
df_train=df_train[:10000]
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
vectorizer=TfidfVectorizer(tokenizer=Okt().morphs)

X = vectorizer.fit_transform(df_train.iloc[:,0])

# Y = df_train.iloc[:,1]
Y = df_train['encoded_target']


c:\Users\uk246\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [65]:
print(X.toarray()[:3])
# sparse(희소) matrix(행열) => np.expand_dims(X.toarray()) dense(밀집) matrix(행렬)
#shape의 차이다. 
# 1. != 1  아니다. 1.0000000... 인거다.

#완성된 x_train
X_train = np.expand_dims(X.toarray(),axis=-1)


[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [67]:
Y  # 5개 클래스 == 다중클래스
# 5개를 분류하는일 다중클래스 분류 문제 multi-classification 
# 두개면 0, 1 == 이진, 이진분류문제 binary-classification

type( Y )  #시리즈객체

# 1,  1    이렇게 되어있는 시리즈를 컴터가 못읽는다.
# 2,  2     그래서 형태를 바꿔줄것이다.
# 3,  3 
# 4,  4

Y.shape  #(10000,)

#1차원 배열로 바꿔주었다.
y = Y.to_numpy()
#리스트 = 자료구조, 데이터 저장하는 구조  type= list()
#넘파이 배열 = 수학적인 행렬, 계산을 할때 사용하는 구조 type= np.array()
y.shape


#타겟 라벨 데이터 차원 맞추기
# y=y_train.to_frame()
# y.shape

# #넘파이 배열로 변환
# y=y.to_numpy()


(10000,)

In [ ]:
print(X_train.shape)  # 5794=> 모델의 벡터차원수를 의미
print(y.shape)
print(X.toarray().shape)


# 전처리가 할것 X,Y 의 타입을 맞춰줌, 차원도 맞춰줌

(8000, 5794, 1)
(10000,)
(10000, 5794)


In [72]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X.toarray(), y, test_size=0.2, random_state=42)

svm_clf = SVC(kernel='linear')

svm_clf.fit(X_train, y_train)

y_pred = svm_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
